In [2]:
from collections import namedtuple
from pathlib import Path
from typing import List

import numpy as np

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR / "2022-16.txt"

def get_inputs(filename:Path=INPUTFILE, example:bool=False) -> List[str]:

    with INPUTFILE.open("r") as file:
        inputs = [line.strip() for line in file.readlines()]

    if example:
        test = [
            "Valve AA has flow rate=0; tunnels lead to valves DD, II, BB",
            "Valve BB has flow rate=13; tunnels lead to valves CC, AA",
            "Valve CC has flow rate=2; tunnels lead to valves DD, BB",
            "Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE",
            "Valve EE has flow rate=3; tunnels lead to valves FF, DD",
            "Valve FF has flow rate=0; tunnels lead to valves EE, GG",
            "Valve GG has flow rate=0; tunnels lead to valves FF, HH",
            "Valve HH has flow rate=22; tunnel leads to valve GG",
            "Valve II has flow rate=0; tunnels lead to valves AA, JJ",
            "Valve JJ has flow rate=21; tunnel leads to valve II",
        ]
        inputs = test

    for i in range(min(8, len(inputs))):
        print(f"{i}: {inputs[i]}")

    return inputs

In [3]:
from typing import Dict
Valve = namedtuple("Valve", ["rate", "neighbors"])

def get_valves(inputs:List[str]) -> Dict[str,Valve]:

    valves = {}
    for line in inputs:
        line = line.split(" ", maxsplit=9)
        name = line[1]
        rate = int(line[4].split("=")[1][:-1])
        neighbors = line[9].split(", ")
        valves[name] = Valve(rate, neighbors)

    print(valves["AA"])
    print(valves["KP"])

    return valves


In [3]:
# Route = namedtuple("Route", ["path", "open", "cost", "income"])
# best_route = Route(tuple(["AA"]), tuple(), 0, 0)
# routes = set([best_route])
# # frozen = set()
# while len(routes) > 0:
#     min_cost = min(map(lambda r:r.cost, routes))
#     for route in list(filter(lambda r:r.cost == min_cost, routes)):
#         valve = route.path[-1]
#         # if there's any value to opening the valve, and it's not already open, and there's time...
#         rate = valves[valve].rate
#         if (rate > 0) and (valve not in route.open) and (30 - route.cost > 1):
#             new_open = tuple(set(route.open)|set([valve]))
#             new_cost = route.cost+1
#             new_income = route.income + rate*(30-new_cost)
#             routes.add(Route(route.path, new_open, new_cost, new_income))
#         neighbors = [neighbor for neighbor in valves[valve].neighbors if neighbor not in route.path]
#         if len(neighbors) > 0:
#             for neighbor in neighbors:
#                 routes.add(Route(tuple(list(route.path)+[neighbor]), route.open, route.cost+1, route.income))
#         else:
#             if route.income > best_route.income:
#                 best_route = route
#         routes.remove(route)

# # max_income = max(map(lambda r:r.income, frozen))
# # print(f"{max_income=}")
# print(best_route)


In [4]:
from typing import Tuple
from collections import defaultdict

def shortest_path(start: str, end: str, valves:Dict[str, Tuple[int, List[str]]]) -> List[str]:

    shortest = []
    paths = defaultdict(list)
    paths[0] = [[start]]
    length = 0
    while len(shortest) == 0:
        for path in paths[length]:
            terminus = valves[path[-1]]
            for neighbor in terminus.neighbors:
                if neighbor not in path:
                    appended = path + [neighbor]
                    if neighbor == end:
                        shortest = appended
                        break
                    paths[length+1].append(appended)
            if len(shortest) == 0:
                break
        length += 1

    return shortest

# print(f"{shortest_path('AA', 'PM', valves)=}")
# print(f"{shortest_path('AA', 'CJ', valves)=}")


In [5]:
from datetime import datetime

inputs = get_inputs()
valves = get_valves(inputs)

unopened = [key for key in valves if valves[key].rate > 0]
print(f"{unopened=}")
tstart = datetime.now()
# options = [shortest_path("AA", valve, valves) for valve in unopened]
options = []
for valve in unopened:
    print(f"{valve}...", end="")
    shortest = shortest_path("AA", valve, valves)
    print(f"{shortest}")
    options.append(shortest)
tfinish = datetime.now()
print(f"{tfinish-tstart} to calculate shortest paths")
# print(f"{options=}")

for path in options:
    print(f"unopened valves in {path} = {len(set(path) & set(unopened))}")

0: Valve AA has flow rate=0; tunnels lead to valves PM, MU, BM, AW, CB
1: Valve AG has flow rate=0; tunnels lead to valves IS, MW
2: Valve AW has flow rate=0; tunnels lead to valves BS, AA
3: Valve BM has flow rate=0; tunnels lead to valves MW, AA
4: Valve BS has flow rate=0; tunnels lead to valves IS, AW
5: Valve BU has flow rate=0; tunnels lead to valves OE, EV
6: Valve BY has flow rate=0; tunnels lead to valves QD, ZF
7: Valve CB has flow rate=0; tunnels lead to valves AA, FX
Valve(rate=0, neighbors=['PM', 'MU', 'BM', 'AW', 'CB'])
Valve(rate=25, neighbors=['YO'])
unopened=['CN', 'EV', 'FB', 'FX', 'HB', 'IS', 'JO', 'KP', 'MW', 'OH', 'QD', 'RD', 'TB', 'ZF', 'ZP']
CN...

: 

: 

In [10]:
from typing import Set
from itertools import combinations

def value_of(path: List[str], time: int, unopened: Set[str], valves:Dict[str, Tuple[int, List[str]]]) -> int:

    assert (terminus := path[-1]) in unopened

    intermediates:Set = set(path) & unopened
    intermediates.remove(terminus)

    best_value = 0
    valves_on = set()
    for count in range(len(intermediates)+1):
        for on in combinations(intermediates, count):
            on = set(on)
            on.add(terminus)
            elapsed = 0
            value = 0
            for valve in path:
                if valve in on:
                    elapsed += 1
                    value += valves[valve].rate * (time - elapsed)
                elapsed += 1
            if value > best_value:
                best_value = value
                valves_on = set(on)

    return best_value, valves_on, elapsed

print(f"{value_of(['AA', 'BM', 'MW', 'NL', 'JO'], 30, set(unopened), valves)}")


(729, {'MW', 'JO'}, 7)


In [11]:
unopened = set([key for key in valves if valves[key].rate > 0])
best_path = ["AA"]
best_value = 0
remaining = 30

while remaining > 0:

    next_path = []
    next_value = 0
    next_on = set()
    next_elapsed = 0

    for option in [shortest_path(best_path[-1], valve, valves) for valve in unopened]:
        value, on, elapsed = value_of(option, remaining, unopened, valves)
        if (value > next_value) and (elapsed <= remaining):
            next_path = option
            next_value = value
            next_on = on
            next_elapsed = elapsed

    best_path += next_path[1:]
    best_value += next_value
    unopened -= next_on
    remaining -= next_elapsed

    if next_elapsed == 0:   # didn't find any valid candidates
        break

    if len(unopened) == 0:  # opened all available useful valves
        break

print(f"{best_path=}")
print(f"{best_value=}") # 1840 is too low
print(f"{remaining=}")

best_path=['AA', 'BM', 'MW', 'NL', 'JO', 'SX', 'EV', 'BU', 'OE', 'FB', 'OE', 'BU', 'EV', 'IU', 'CN', 'HC', 'HB']
best_value=1840
remaining=6
